In [13]:
import requests, pprint, http.client, json

In [29]:
#   movies.json

conn = http.client.HTTPSConnection("api.themoviedb.org")

payload = "{}"
dataObject = []
for i in range(20):
   conn.request("GET", "/3/movie/popular?page="+f"{i+1}"+"&language=ko-kr&api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
   res = conn.getresponse()
   data = res.read()
   dataObject += json.loads(data)['results']

with open('movies1.json', 'w', encoding='utf-8') as f:
        json.dump(dataObject, f, ensure_ascii=False, indent=4)

In [30]:
movie_id = []
for i in range(len(dataObject)):
    movie_id.append(dataObject[i]["id"])

In [ ]:
#  actor1.json

conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"
raw = []
dataObject = []

for i in movie_id:
    conn.request("GET", "/3/movie/"+f"{i}"+"/credits?api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
    res = conn.getresponse()
    data = json.loads(res.read())
    dataObject += [{"movie_id" : i}]
    
    for j in range(3):
        try:
            raw += data['cast']
            dataObject.append(raw[j])
        except:
            pass
        raw = []
    with open('actor1.json', 'w', encoding='utf-8') as f:
        json.dump(dataObject, f, ensure_ascii=False, indent=4)

In [ ]:
conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"
raw = []
dataObject = []
dataDetail = []

for i in movie_id:
    conn.request("GET", "/3/movie/"+f"{i}"+"/credits?api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
    res = conn.getresponse()
    data = json.loads(res.read())
    dataObject += [{"movie_id" : i}]
    
    for j in range(3):
        try:
            raw += data['cast']
            dataObject.append(raw[j])
        except:
            pass
        raw = []
    dataDetail += [dataObject]
    with open('actor2.json', 'w', encoding='utf-8') as f:
        json.dump(dataObject, f, ensure_ascii=False, indent=4)

In [31]:
#  actor.json 완성작

conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"
raw = []
dataObject = []

for i in movie_id:
    conn.request("GET", "/3/movie/"+f"{i}"+"/credits?api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
#     conn.request("GET", "/3/movie/"+f"{i}"+"/credits?api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
    res = conn.getresponse()
    data = json.loads(res.read())
    for j in range(min(3, len(data['cast']))):
        dataDetail = {}
        dataDetail['pk'] = data['cast'][j]['id']
        dataDetail['model'] = 'movies.actor'
        dataDetail['fields'] = {
            'name': data['cast'][j]['name'],
            'img': data['cast'][j]['profile_path'],
            'acted': i
        }
        dataObject.append(dataDetail)

    with open('actor.json', 'w', encoding='utf-8') as f:
        json.dump(dataObject, f, ensure_ascii=False, indent=4)

KeyError: 'cast'

In [28]:
#  director.json 완성작

conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"
dataObject = []


for i in movie_id:
    conn.request("GET", "/3/movie/"+f"{i}"+"/credits?api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
    res = conn.getresponse()
    data = json.loads(res.read())
    for j in range(len(data['crew'])):
        if data['crew'][j]['job'] == 'Director':
            dataDetail = {}
            dataDetail['pk'] = data['crew'][j]['id']
            dataDetail['model'] = 'movies.director'
            dataDetail['fields'] = {
             'name': data['crew'][j]['name'],
            'job': data['crew'][j]['job'],
            'img': data['crew'][j]['profile_path'],
            'directed': i
            }
            dataObject.append(dataDetail)
            
    with open('director.json', 'w', encoding='utf-8') as f:
        json.dump(dataObject, f, ensure_ascii=False, indent=4)

In [ ]:
import requests, http.client, json

#  genres.json 완성작

conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"
dataObject = []
conn.request("GET", "/3/genre/movie/list?language=ko-kr&api_key=<<api_key>>", payload)
res = conn.getresponse()
data = res.read()
dataObject += json.loads(data)['genres']

with open('genres.json', 'w', encoding='utf-8') as f:
    json.dump(dataObject, f, ensure_ascii=False, indent=4)

In [1]:
import csv
import codecs
import json
import os
import re
import sys


# Options
ENCODING = 'utf-8'
CSV_DELIMITER = ','

# Check arguments
try:
  script, input_name, model_name = sys.argv
except ValueError:
  print('\nRun via:\n\n%s input_name model_name' % sys.argv[0])
  print('\ne.g. %s airport.csv app_airport.Airport' % sys.argv[0])
  print('\nNote: input_name should be a path relative to where this script is.')
  sys.exit()

# Compute file paths and names
in_file = os.path.dirname(__file__) + input_name
if len(re.findall('\.csv$', input_name)) != 0:
  output_name = re.sub('\.csv$', '.json', input_name)
else:
  output_name += '.json'
out_file = os.path.dirname(__file__) + output_name

# Convert
print('Converting %s from CSV to JSON as %s' % (in_file, out_file))
with codecs.open(in_file, 'r', encoding=ENCODING) as f:

  reader = csv.reader(f, delimiter=CSV_DELIMITER)

  header_row = []
  entries = []

  for row in reader:

    if not header_row:
      header_row = row
      continue

    pk = row[0]
    model = model_name
    fields = {}
    for i in range(len(row) - 1):
      active_field = row[i+1] if row[i+1] != '' else '0'

      # convert numeric strings into actual numbers by converting to either
      # int or float
      if active_field.isdigit():
        try:
          new_number = int(active_field)
        except ValueError:
          new_number = float(active_field)
        fields[header_row[i+1]] = new_number
      else:
        fields[header_row[i+1]] = active_field.strip()

    row_dict = {}
    row_dict['pk'] = int(pk)
    row_dict['model'] = model_name
    row_dict['fields'] = fields
    entries.append(row_dict)

f.close()

with open(out_file, 'w') as fo:
    fo.write('%s' % json.dumps(entries, indent=4, ensure_ascii=False))
fo.close()

NameError: name '__file__' is not defined